In [1]:
import pandas as pd
movies = pd.read_csv('data/preprocessed_data.csv')

In [2]:
movies.head(1)

,Unnamed: 0,id,title,genres,keywords,overview,cast,crew
0,0,19995,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['culture clash', 'future', 'space war', 'spac...","In the 22nd century, a paraplegic Marine is di...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",['James Cameron']


In [3]:
import ast
def convert(temp):
    l = []
    for i in ast.literal_eval(temp):
        l.append(i)
    return l

In [4]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [5]:
movies['genres'] = movies['genres'].apply(convert)

In [6]:
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert)
movies['crew'] = movies['crew'].apply(convert)

In [7]:
def to_lower(l):
    k=[]
    for i in l:
        k.append(i.lower())
    return k

In [8]:
movies['tags'] = movies['genres']+movies['keywords']+movies['overview']+movies['cast']+movies['crew']

In [9]:
movies = movies[['id','title','tags']]

In [10]:
movies['tags'] = movies['tags'].apply(to_lower)

In [11]:
movies['tags']

0       [action, adventure, fantasy, science fiction, ...
1       [adventure, fantasy, action, ocean, drug abuse...
2       [action, adventure, crime, spy, based on novel...
3       [action, crime, drama, thriller, dc comics, cr...
4       [action, adventure, science fiction, based on ...
                              ...                        
4801    [action, crime, thriller, united states–mexico...
4802    [comedy, romance, a, newlywed, couple's, honey...
4803    [comedy, drama, romance, tv movie, date, love ...
4804    [when, ambitious, new, york, attorney, sam, is...
4805    [documentary, obsession, camcorder, crush, dre...
Name: tags, Length: 4806, dtype: object

In [12]:
movies['tags'] = movies['tags'].apply(lambda x:' '.join(x))

In [13]:
movies.head()

,id,title,tags
0,19995,Avatar,action adventure fantasy science fiction cultu...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drug abuse exot...
2,206647,Spectre,action adventure crime spy based on novel secr...
3,49026,The Dark Knight Rises,action crime drama thriller dc comics crime fi...
4,49529,John Carter,action adventure science fiction based on nove...


In [14]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [15]:
def stem(l):
    k=[]
    for i in l.split():
        k.append(ps.stem(i))
    return ' '.join(k)

In [16]:
movies['tags'] = movies['tags'].apply(stem)

In [17]:
movies.head()

,id,title,tags
0,19995,Avatar,action adventur fantasi scienc fiction cultur ...
1,285,Pirates of the Caribbean: At World's End,adventur fantasi action ocean drug abus exot i...
2,206647,Spectre,action adventur crime spi base on novel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dc comic crime fig...
4,49529,John Carter,action adventur scienc fiction base on novel m...


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [19]:
vector = cv.fit_transform(movies['tags']).toarray()

In [20]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [22]:
similarity

array([[1.        , 0.06692745, 0.06554735, ..., 0.04442617, 0.03244428,
        0.        ],
       [0.06692745, 1.        , 0.04167571, ..., 0.03766218, 0.        ,
        0.01731099],
       [0.06554735, 0.04167571, 1.        , ..., 0.01844278, 0.0808122 ,
        0.        ],
       ...,
       [0.04442617, 0.03766218, 0.01844278, ..., 1.        , 0.03651484,
        0.03064257],
       [0.03244428, 0.        , 0.0808122 , ..., 0.03651484, 1.        ,
        0.06713451],
       [0.        , 0.01731099, 0.        , ..., 0.03064257, 0.06713451,
        1.        ]])

In [23]:
movie_posters = pd.read_csv('data/movie_posters.csv')
def get_poster(id):
    ind = movie_posters[movie_posters['movie_id'] == id].index[0]
    return movie_posters.poster_link[ind]

In [24]:
def recommended(movie_name):
    movie_id = movies[movies['title']==movie_name].index[0]
    mat = similarity[movie_id]
    l = []
    p = []
    mat = list(enumerate(mat))
    mat = sorted(mat,reverse=True, key = lambda x:x[1])[1:6]
    
    for i in mat:
        l.append(movies.iloc[i[0]].title)
        p.append(get_poster(movies.iloc[i[0]].id))
    return l,p

In [25]:
recommended('The Avengers')

(['Avengers: Age of Ultron',
  'Captain America: Civil War',
  'Iron Man 3',
  'X-Men: Apocalypse',
  'Iron Man'],
 ['https://image.tmdb.org/t/p/w500/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg',
  'https://image.tmdb.org/t/p/w500/rAGiXaUfPzY7CDEyNKUofk3Kw2e.jpg',
  'https://image.tmdb.org/t/p/w500/qhPtAc1TKbMPqNvcdXSOn9Bn7hZ.jpg',
  'https://image.tmdb.org/t/p/w500/2mtQwJKVKQrZgTz49Dizb25eOQQ.jpg',
  'https://image.tmdb.org/t/p/w500/78lPtwv72eTNqFW9COBYI0dWDJa.jpg'])

In [76]:
movies.to_csv('data/final_data.csv')